# **Imports**

In [105]:
import pandas as pd # pip install pandas
from difflib import SequenceMatcher

Football Matches of Spanish League
https://www.kaggle.com/datasets/ricardomoya/football-matches-of-spanish-league

Premier League Matches 1993-2023
https://www.kaggle.com/datasets/evangower/premier-league-matches-19922022

Turkish Super League Matches (1959-2021)
https://www.kaggle.com/datasets/faruky/turkish-super-league-matches-19592020

France Football from 1950 to 2022
https://www.kaggle.com/datasets/simonherv/france-football-from-1950-to-2022

—
Merge Both

Football | Bundesliga Seasons 2005/06 - 2022/23
https://www.kaggle.com/datasets/oles04/bundesliga-seasons

Bundesliga Results 1993-2018
https://www.kaggle.com/datasets/thefc17/bundesliga-results-19932018

Import from Github

In [106]:
Bundesliga_1993 = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/Bundesliga%20Results%201993-2018.csv')
Bundesliga_2005 = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/Football%20%7C%20Bundesliga%20Seasons%202005%3A06%20-%202022%3A23.csv')
France_League = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/France%20Football%20from%201950%20to%202022.csv')
Turkish_League = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/Turkish%20Super%20League%20Matches%20(1959-2021).csv')
Premier_League = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/Premier%20League%20Matches%201993-2023.csv')
Spanish_League = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/Spanish_League%201970-2017.csv')
Spanish_League_1995 = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/LaLiga_1995-2023.csv')

Import Locally Jupyter

In [107]:
#Spanish_League = pd.read_csv('Spanish_League 1970-2017.csv')
#Premier_League = pd.read_csv('Premier League Matches 1993-2023.csv')
#Turkish_League = pd.read_csv('Turkish Super League Matches (1959-2021).csv')
#France_League = pd.read_csv('France Football from 1950 to 2022.csv')
##Bundesliga_2005 = pd.read_csv('Football | Bundesliga Seasons 2005/06 - 2022/23.csv')
#Bundesliga_1993 = pd.read_csv('Bundesliga Results 1993-2018.csv')

# **Global Schema**

We need to define a global schema for all datasets

Global schema         ["Date", "HomeTeam", "AwayTeam", "HomeGoals", "AwayGoals", "Winner", "League"]

# *Bundesliga_1993* 

In [108]:
Bundesliga_1993.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Season'],
      dtype='object')

In [110]:
# Dropping specific columns ('Div', 'Season', 'HTHG', 'HTAG', 'HTR') from the 'Bundesliga_1993' DataFrame
Bundesliga_1993_new = Bundesliga_1993.drop(columns=['Div', 'Season', 'HTHG', 'HTAG', 'HTR'], axis=1)

# Renaming selected columns of the newly created DataFrame ('Bundesliga_1993_new') for better understanding and consistency
Bundesliga_1993_new = Bundesliga_1993_new.rename(columns={'Date': 'Date', 'HomeTeam': 'HomeTeam', 'AwayTeam': 'AwayTeam', 'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals', 'FTR': 'Winner'})

# Converting the 'Date' column to datetime format and ensuring a consistent date format ('%Y-%m-%d')
Bundesliga_1993_new['Date'] = pd.to_datetime(Bundesliga_1993_new['Date'], format='mixed').dt.strftime('%Y-%m-%d')

# Adding a 'League' column with the value 'Bundesliga' for each row to indicate the league name
Bundesliga_1993_new['League'] = 'Bundesliga'

Bundesliga_1993_new.head(2)

,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Winner,League
0,1993-07-08,Bayern Munich,Freiburg,3,1,H,Bundesliga
1,1993-07-08,Dortmund,Karlsruhe,2,1,H,Bundesliga


# *Bundesliga_2005*

In [81]:
Bundesliga_2005.columns

Index(['Unnamed: 0', 'MATCH_DATE', 'LEAGUE_NAME', 'SEASON', 'LEAGUE',
       'FINISHED', 'LOCATION', 'VIEWER', 'MATCHDAY', 'MATCHDAY_NR',
       'HOME_TEAM_ID', 'HOME_TEAM_NAME', 'HOME_TEAM', 'HOME_ICON',
       'AWAY_TEAM_ID', 'AWAY_TEAM_NAME', 'AWAY_TEAM', 'AWAY_ICON',
       'GOALS_HOME', 'GOALS_AWAY', 'DRAW', 'WIN_HOME', 'WIN_AWAY'],
      dtype='object')

In [82]:
Bundesliga_2005_new = Bundesliga_2005.loc[:, ['MATCH_DATE', 'HOME_TEAM_NAME', 'AWAY_TEAM_NAME', 'GOALS_HOME', 'GOALS_AWAY']]
Bundesliga_2005_new = Bundesliga_2005_new.rename(columns={'MATCH_DATE': 'Date', 'HOME_TEAM_NAME': 'HomeTeam', 'AWAY_TEAM_NAME': 'AwayTeam', 'GOALS_HOME': 'HomeGoals', 'GOALS_AWAY': 'AwayGoals'})
Bundesliga_2005_new['Winner'] = ['H' if x > y else 'A' if x < y else 'D' for x, y in zip(Bundesliga_2005_new['HomeGoals'], Bundesliga_2005_new['AwayGoals'])]
Bundesliga_2005_new['Date'] = pd.to_datetime(Bundesliga_2005_new['Date'], format='mixed').dt.strftime('%Y-%m-%d')
Bundesliga_2005_new['League'] = 'Bundesliga'
Bundesliga_2005_new.head(2)

,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Winner,League
0,2005-08-05,FC Bayern München,Borussia Mönchengladbach,3,0,H,Bundesliga
1,2005-08-06,1. FC Köln,1. FSV Mainz 05,1,0,H,Bundesliga


In [83]:
# Team names are not consistent in BL1 and BL2
Bundesliga_1993_new['HomeTeam'].unique(), Bundesliga_2005_new['HomeTeam'].unique()

(array(['Bayern Munich', 'Dortmund', 'Duisburg', 'FC Koln', 'Hamburg',
        'Leipzig', "M'Gladbach", 'Wattenscheid', 'Werder Bremen',
        'Dresden', 'Ein Frankfurt', 'Freiburg', 'Kaiserslautern',
        'Karlsruhe', 'Leverkusen', 'Nurnberg', 'Schalke 04', 'Stuttgart',
        'Uerdingen', 'Bochum', 'Munich 1860', "M'gladbach",
        'Hansa Rostock', 'St Pauli', 'Dusseldorf', 'Bielefeld', 'Hertha',
        'Wolfsburg', 'Ulm', 'Unterhaching', 'Cottbus', 'Hannover', 'Mainz',
        'Aachen', 'Hoffenheim', 'Augsburg', 'Greuther Furth',
        'Fortuna Dusseldorf', 'Braunschweig', 'Paderborn', 'Darmstadt',
        'Ingolstadt', 'RB Leipzig'], dtype=object),
 array(['FC Bayern München', '1. FC Köln', 'MSV Duisburg', 'Hamburger SV',
        'VfL Wolfsburg', 'Werder Bremen', 'Hannover 96', 'FC Schalke 04',
        'Eintracht Frankfurt', 'Bayer Leverkusen',
        'Borussia Mönchengladbach', 'Borussia Dortmund', 'Hertha BSC',
        '1. FC Kaiserslautern', '1. FC Nürnberg', 'Armin

# **Matching Team Names in Bundesliga Datasets**

In [84]:
def levenshtein_distance(s1, s2):
    return SequenceMatcher(None, s1, s2).ratio()

# find the best match for each team in Bundesliga_1993
teams_2005 = [team for team in Bundesliga_2005_new['HomeTeam'].unique() if len(team) > 1]
for t1 in Bundesliga_1993_new['HomeTeam'].unique():
    best_match = ''
    best_match_score = 0
    for t2 in teams_2005:
        score = levenshtein_distance(t1.lower(), t2.lower())
        if score > best_match_score:
            best_match = t2
            best_match_score = score

    if best_match != '':
        # change each team name in Bundesliga_1993 to the best match from Bundesliga_2005
        Bundesliga_1993_new['HomeTeam'] = Bundesliga_1993_new['HomeTeam'].replace(t1, best_match)
        Bundesliga_1993_new['AwayTeam'] = Bundesliga_1993_new['AwayTeam'].replace(t1, best_match)

        # remove the best match from Bundesliga_2005 to avoid matching it again
        teams_2005.remove(best_match)

In [85]:
Bundesliga_1993_new.head(10)

,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Winner,League
0,1993-07-08,FC Bayern München,SC Freiburg,3,1,H,Bundesliga
1,1993-07-08,Borussia Dortmund,Karlsruher SC,2,1,H,Bundesliga
2,1993-07-08,MSV Duisburg,FC Energie Cottbus,2,2,D,Bundesliga
3,1993-07-08,1. FC Köln,1. FC Kaiserslautern,0,2,A,Bundesliga
4,1993-07-08,Hamburger SV,1. FC Nürnberg,5,2,H,Bundesliga
5,1993-07-08,RB Leipzig,Bayer Leverkusen,3,3,D,Bundesliga
6,1993-07-08,Borussia Mönchengladbach,Eintracht Frankfurt,0,4,A,Bundesliga
7,1993-07-08,Eintracht Braunschweig,FC Schalke 04,3,0,H,Bundesliga
8,1993-07-08,Werder Bremen,VfB Stuttgart,5,1,H,Bundesliga
9,1993-08-14,Bayer Leverkusen,MSV Duisburg,0,1,A,Bundesliga


# *France_League*

In [86]:
France_League.columns

Index(['id', 'date', 'local', 'visiteur', 'ligue', 'saison', 'local_score',
       'visiteur_score', 'resultat'],
      dtype='object')

In [87]:
France_League_new = France_League.loc[:, ['date', 'local', 'visiteur', 'local_score', 'visiteur_score']]
France_League_new = France_League_new.rename(columns={'date': 'Date', 'local': 'HomeTeam', 'visiteur': 'AwayTeam', 'local_score': 'HomeGoals', 'visiteur_score': 'AwayGoals'})
France_League_new['Winner'] = ['H' if x > y else 'A' if x < y else 'D' for x, y in zip(France_League_new['HomeGoals'], France_League_new['AwayGoals'])]
France_League_new['Date'] = pd.to_datetime(France_League_new['Date'], format='mixed').dt.strftime('%Y-%m-%d')
France_League_new['League'] = 'Ligue 1'
France_League_new.head(2)

,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Winner,League
0,2023-01-29,OGC Nice,LOSC Lille,1,0,H,Ligue 1
1,2023-01-29,AJ Auxerre,Montpellier HSC,0,2,A,Ligue 1


# *Premier_League*

In [88]:
Premier_League.columns

Index(['Season_End_Year', 'Wk', 'Date', 'Home', 'HomeGoals', 'AwayGoals',
       'Away', 'FTR'],
      dtype='object')

In [89]:
Premier_League_new = Premier_League.loc[:, ['Date', 'Home', 'Away', 'HomeGoals', 'AwayGoals', 'FTR']]
Premier_League_new = Premier_League_new.rename(columns={'Home': 'HomeTeam', 'Away': 'AwayTeam', 'FTR': 'Winner'})
Premier_League_new['Date'] = pd.to_datetime(Premier_League_new['Date'], format='mixed').dt.strftime('%Y-%m-%d')
Premier_League_new['League'] = 'Premier League'
Premier_League_new.head(2)

,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Winner,League
0,1992-08-15,Coventry City,Middlesbrough,2,1,H,Premier League
1,1992-08-15,Leeds United,Wimbledon,2,1,H,Premier League


# *Spanish_League*

In [90]:
Spanish_League.columns

Index(['id', 'season', 'division', 'round', 'localTeam', 'visitorTeam',
       'localGoals', 'visitorGoals', 'date', 'timestamp'],
      dtype='object')

In [91]:
Spanish_League_new = Spanish_League.loc[:, ['date', 'localTeam', 'visitorTeam', 'localGoals', 'visitorGoals']]
Spanish_League_new = Spanish_League_new.rename(columns={'date': 'Date', 'localTeam': 'HomeTeam', 'visitorTeam': 'AwayTeam', 'localGoals': 'HomeGoals', 'visitorGoals': 'AwayGoals'})
Spanish_League_new['Winner'] = ['H' if x > y else 'A' if x < y else 'D' for x, y in zip(Spanish_League_new['HomeGoals'], Spanish_League_new['AwayGoals'])]
Spanish_League_new['Date'] = pd.to_datetime(Spanish_League_new['Date'], format='mixed').dt.strftime('%Y-%m-%d')
Spanish_League_new['League'] = 'La Liga'
Spanish_League_new.head(2)

,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Winner,League
0,1970-12-09,Atletico de Bilbao,Barcelona,1,1,D,La Liga
1,1970-12-09,Las Palmas,Atletico de Madrid,1,1,D,La Liga


# **Spanish League 1995-2023**

In [92]:
Spanish_League_1995.columns

Index(['Season', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR'],
      dtype='object')

In [93]:
Spanish_League_1995_new = Spanish_League_1995.loc[:, ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG']]
Spanish_League_1995_new = Spanish_League_1995_new.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})
Spanish_League_1995_new['Winner'] = ['H' if x > y else 'A' if x < y else 'D' for x, y in zip(Spanish_League_1995_new['HomeGoals'], Spanish_League_1995_new['AwayGoals'])]
Spanish_League_1995_new['Date'] = pd.to_datetime(Spanish_League_1995_new['Date'], format='mixed').dt.strftime('%Y-%m-%d')
Spanish_League_1995_new['League'] = 'La Liga'
Spanish_League_1995_new.head(2)

,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Winner,League
0,1995-02-09,La Coruna,Valencia,3,0,H,La Liga
1,1995-02-09,Sp Gijon,Albacete,3,0,H,La Liga


In [94]:
# find the best match for each team in Spanish_League
teams_1995 = [team for team in Spanish_League_1995_new['HomeTeam'].unique() if len(team) > 1]
for t1 in Spanish_League_new['HomeTeam'].unique():
    best_match = ''
    best_match_score = 0
    for t2 in teams_1995:
        score = levenshtein_distance(t1.lower(), t2.lower())
        if score > best_match_score:
            best_match = t2
            best_match_score = score

    if best_match != '':
        # change each team name in Spanish_League to the best match from Spanish_League_1995
        Spanish_League_new['HomeTeam'] = Spanish_League_new['HomeTeam'].replace(t1, best_match)
        Spanish_League_new['AwayTeam'] = Spanish_League_new['AwayTeam'].replace(t1, best_match)

        # remove the best match from teams_1995 to avoid matching it again
        teams_1995.remove(best_match)

# *Turkish_League*

In [95]:
Turkish_League.columns

Index(['Date', 'Season', 'Week', 'home', 'visitor', 'FT', 'hgoal', 'vgoal',
       'division', 'tier', 'totgoal', 'goaldiff', 'result', 'HT', 'hgoal_half',
       'vgoal_half', 'half_totgoal', 'half_goaldiff', 'result_half', 'fans',
       'neutral', 'home_red_card', 'visitor_red_card'],
      dtype='object')

In [96]:
Turkish_League_new = Turkish_League.loc[:, ['Date', 'home', 'visitor', 'hgoal', 'vgoal', 'result']]
Turkish_League_new = Turkish_League_new.rename(columns={'home': 'HomeTeam', 'visitor': 'AwayTeam', 'hgoal': 'HomeGoals', 'vgoal': 'AwayGoals', 'result': 'Winner'})
Turkish_League_new['Date'] = pd.to_datetime(Turkish_League_new['Date'], format='mixed').dt.strftime('%Y-%m-%d')
Turkish_League_new['League'] = 'Turkish Super League'
Turkish_League_new.head(2)

,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Winner,League
0,1959-02-21,Genclerbirligi,Adalet,1,1,D,Turkish Super League
1,1959-02-21,Izmirspor,Beykoz 1908,2,1,H,Turkish Super League


# **Concatenate All Datasets**

In [97]:
#concatenate DataFrames (Bundesliga_1993_new, Bundesliga_2005_new, France_League_new, Premier_League_new, Spanish_League_new, Turkish_League_new)
data = pd.concat([Bundesliga_1993_new, Bundesliga_2005_new, France_League_new, Premier_League_new, Spanish_League_new, Spanish_League_1995_new, Turkish_League_new], ignore_index=True)

#checks whether the length of the concatenated DataFrame data is equal to the sum of the lengths of all the individual DataFrames
assert len(data) == len(Bundesliga_1993_new) + len(Bundesliga_2005_new) + len(France_League_new) + len(Premier_League_new) + len(Spanish_League_new) + len(Turkish_League_new) + len(Spanish_League_1995_new)

In [102]:
# add year column for easier queries
data['Date'] = pd.to_datetime(data.Date, format='%Y-%m-%d')
data['Year'] = data.Date.dt.year
data

,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Winner,League,Year
0,1993-07-08,FC Bayern München,SC Freiburg,3,1,H,Bundesliga,1993
1,1993-07-08,Borussia Dortmund,Karlsruher SC,2,1,H,Bundesliga,1993
2,1993-07-08,MSV Duisburg,FC Energie Cottbus,2,2,D,Bundesliga,1993
3,1993-07-08,1. FC Köln,1. FC Kaiserslautern,0,2,A,Bundesliga,1993
4,1993-07-08,Hamburger SV,1. FC Nürnberg,5,2,H,Bundesliga,1993
...,...,...,...,...,...,...,...,...
202327,2021-10-01,Caykur Rizespor,Denizlispor,1,1,D,Turkish Super League,2021
202328,2021-10-01,Hatayspor,Besiktas,2,2,D,Turkish Super League,2021
202329,2021-11-01,BB Erzurumspor,Fenerbahce,0,3,A,Turkish Super League,2021
202330,2021-11-01,Alanyaspor,Kasimpasa,1,2,A,Turkish Super League,2021


In [71]:
data.isna().sum() #display the count of missing values

Date         0
HomeTeam     0
AwayTeam     0
HomeGoals    0
AwayGoals    0
Winner       0
League       0
dtype: int64

# **Remove Duplicates**

In [72]:
# Checking for duplicates in the updated data
duplicates_check = data[data.duplicated(subset=["Date", "HomeTeam", "AwayTeam", "HomeGoals", "AwayGoals", "Winner", "League"], keep=False)]

# Printing the duplicates (if any)
if len(duplicates_check) == 0:
    print("No duplicates found")
else:
    print(f"There are {len(duplicates_check)} duplicates")


# Remove duplicates 
data_no_duplicates = data.drop_duplicates(subset=["Date", "HomeTeam", "AwayTeam", "HomeGoals", "AwayGoals", "Winner", "League"], keep='first')

# Checking for duplicates in the updated data
duplicates_check = data_no_duplicates[data_no_duplicates.duplicated(subset=["Date", "HomeTeam", "AwayTeam", "HomeGoals", "AwayGoals", "Winner", "League"], keep=False)]

# Printing the duplicates (if any)
if len(duplicates_check) == 0:
    print("No duplicates found after removing duplicates")
else:
    print("Duplicates still exist after removing duplicates")
    print(duplicates_check)

There are 11258 duplicates
No duplicates found after removing duplicates


# **Download the CSV File**

In [75]:
# Assuming 'data_no_duplicates' is the DataFrame you want to save
# Replace 'file_path' with the path where you want to save the CSV file along with the desired file name

file_path = 'data/data_no_duplicates.csv'  # Modify 'path_to_save' and file name as needed

data_no_duplicates.to_csv(file_path, index=False)
